# Data cleaning for SEER 1y incidence

In [2]:
import numpy as np
import pandas as pd
import os
os.getcwd() 

'c:\\repos\\pdac-calibration\\notebooks'

In [15]:
seer_inc = pd.read_csv("../data/pdac_seer17_incidence_1y.csv")
seer_inc.iloc[:5]

,Year Dx (2000-2021 group only) for DevCan,Age recode with single ages and 85+,Combined Summary Stage (2004+),Crude Rate,Count,Population
0,2000-2021,00 years,In situ,0.000000,0,23639727
1,2000-2021,00 years,Localized,0.000000,0,23639727
2,2000-2021,00 years,Regional,0.000000,0,23639727
3,2000-2021,00 years,Distant,0.000000,0,23639727
4,2000-2021,00 years,NaN,0.000000,0,23639727


In [16]:
seer_inc['Age recode with single ages and 85+'].unique()

array(['00 years', '01 years', '02 years', '03 years', '04 years',
       '05 years', '06 years', '07 years', '08 years', '09 years',
       '10 years', '11 years', '12 years', '13 years', '14 years',
       '15 years', '16 years', '17 years', '18 years', '19 years',
       '20 years', '21 years', '22 years', '23 years', '24 years',
       '25 years', '26 years', '27 years', '28 years', '29 years',
       '30 years', '31 years', '32 years', '33 years', '34 years',
       '35 years', '36 years', '37 years', '38 years', '39 years',
       '40 years', '41 years', '42 years', '43 years', '44 years',
       '45 years', '46 years', '47 years', '48 years', '49 years',
       '50 years', '51 years', '52 years', '53 years', '54 years',
       '55 years', '56 years', '57 years', '58 years', '59 years',
       '60 years', '61 years', '62 years', '63 years', '64 years',
       '65 years', '66 years', '67 years', '68 years', '69 years',
       '70 years', '71 years', '72 years', '73 years', '74 yea

In [28]:
# Pivot the data
pivot_df = seer_inc.pivot(
    index='Age recode with single ages and 85+', 
    columns='Combined Summary Stage (2004+)', 
    values='Crude Rate'
)

pivot_df.columns = [f"{col} Rate" for col in pivot_df.columns]
pivot_df.reset_index(inplace=True)
pivot_df = pivot_df[pivot_df['Age recode with single ages and 85+'] != 'Unknown']
pivot_df['Age'] = pivot_df['Age recode with single ages and 85+'].str[:2].astype(int)
pivot_df['Localized Rate'] = pivot_df['Localized Rate'].astype(float)
pivot_df['Regional Rate'] = pivot_df['Regional Rate'].astype(float)
pivot_df['Distant Rate'] = pivot_df['Distant Rate'].astype(float)
pivot_df['Unknown/unstaged Rate'] = pivot_df['Unknown/unstaged Rate'].astype(float)

In [29]:
pivot_df.drop(['nan Rate', 'In situ Rate', 'Blank(s) Rate', 'Age recode with single ages and 85+'], axis=1, inplace=True)

In [30]:
pivot_df.head()

,Distant Rate,Localized Rate,Regional Rate,Unknown/unstaged Rate,Age
0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,4


Redistribute unstaged cancer incidence over local, regional, and distant

In [31]:
pivot_df['Distant Rate'].iloc[4]

0.0

In [32]:
pivot_df['LRD Rate'] = pivot_df['Localized Rate'] + pivot_df['Regional Rate'] + pivot_df['Distant Rate']
pivot_df['LRDU Rate'] = pivot_df['Localized Rate'] + pivot_df['Regional Rate'] + pivot_df['Distant Rate'] + pivot_df['Unknown/unstaged Rate']
pivot_df['pLocal'] = pivot_df['Localized Rate'] / pivot_df['LRD Rate']
pivot_df['pRegional'] = pivot_df['Regional Rate'] / pivot_df['LRD Rate']
pivot_df['pDistant'] = pivot_df['Distant Rate'] / pivot_df['LRD Rate']
pivot_df['LocalU'] = pivot_df['Localized Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pLocal']
pivot_df['RegionalU'] = pivot_df['Regional Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pRegional']
pivot_df['DistantU'] = pivot_df['Distant Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pDistant']

In [33]:
pivot_df = pivot_df[pivot_df['Age']>=20]
pivot_df = pivot_df[pivot_df['Age']<85]

In [34]:
pivot_df.to_csv("../data/seer_incidence_1y.csv")